In [ ]:
# Importamos las librerías necesarias de transformers, torch y accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

In [ ]:
# Definimos el dispositivo: 'cuda' si se dispone de GPU, de lo contrario 'cpu'
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Definimos tu token de Hugging Face directamente en el código
hf_token = "YOUR_TOKEN_KEY"

In [ ]:
# Importamos la función de login desde huggingface_hub
from huggingface_hub import login

In [ ]:
# Autenticamos usando el token proporcionado de Hugging Face
login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Definimos el nombre del modelo que vamos a cargar desde Hugging Face
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
# Cargamos el modelo utilizando el nombre del modelo y el token para autenticación
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Cargamos el tokenizer utilizando el nombre del modelo y el token para autenticación
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
# Definimos el texto de entrada para el modelo
text = ("<s>[INST] Below is a conversation transcript [/INST]"
        "Your credit card has been stolen, and you need to contact us to resolve the issue. "
        "We will help you protect your information and prevent further fraud.[/s] "
        "[INST] Analyze the conversation and determine if it's fraudulent or legitimate. [/INST]")

In [ ]:
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

In [ ]:
model_inputs = encodeds.to(device)

In [ ]:
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [ ]:
# Generamos la salida del modelo usando los tokens de entrada
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# Imprimimos la salida decodificada
print(decoded[0])

<s> [INST] Below is a conversation transcript [/INST]Your credit card has been stolen, and you need to contact us to resolve the issue. We will help you protect your information and prevent further fraud.[/s] [INST] Analyze the conversation and determine if it's fraudulent or legitimate. [/INST] The conversation appears to be legitimate. It involves a customer whose credit card has been stolen and reaching out to a company for assistance in resolving the issue and protecting their information from further fraud. This type of scenario is commonly encountered in customer service interactions.</s>
